<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:


      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-28 09:46:15
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: -8.81 K (-0.06%)
Current PnL: -19.11 L (-12.52%)
CY Booked + Current PnL: -7.01 L (-4.59%)
-------------------
Total profit:  2.54 L
Total loss:  -21.65 L
-------------------
Total Booked + Current PnL: 19.97 L (15.9%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.01 L (59.98%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.91,74.0,X-LC,3.85,232580.0,22214.0,6559.0,0.46,10.56,2.82,13.68,37.0,3.39,1.60,28.83,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,99.36,57.0,M-SC,1.97,85903.0,-14874.0,14964.0,1.36,-14.76,17.42,0.09,245.0,-0.99,0.59,12.86,OX40N,NTT,DURABLES
42,ITC,452.0,-36.23,68.0,X-LC,1.51,205747.0,5609.0,15287.0,0.02,2.80,7.43,10.44,4.0,0.37,1.42,8.52,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-9.24,44.0,X-MC,2.99,89625.0,-1053.0,15962.0,0.35,-1.16,17.81,16.44,101.0,-0.07,0.62,18.78,X40,ATH,PHARMA
86,VOLTAS,1530.0,1.58,54.0,X-MC,3.56,213090.0,21348.0,16408.0,-1.11,11.13,7.70,19.69,99.0,1.30,1.47,18.69,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.0,-43.25,57.0,X-SC,38.26,56097.0,-8909.0,159837.0,5.30,-13.70,284.93,232.18,198.0,-0.06,0.39,9.42,XY24,NTT,MISC
39,INDUSINDBK,1800.0,265.55,64.0,L-MC,9.28,44888.0,-32318.0,57713.0,2.27,-41.86,128.57,32.89,258.0,-0.56,0.31,23.62,XR,NTT,BANKS
75,TATAMOTORS,1065.0,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
79,TTKPRESTIG,770.0,99.36,57.0,M-SC,1.97,85903.0,-14874.0,14964.0,1.36,-14.76,17.42,0.09,245.0,-0.99,0.59,12.86,OX40N,NTT,DURABLES
77,TITAGARH,1548.0,-3.20,56.0,H-SC,2.88,206750.0,-35427.0,146193.0,1.22,-14.63,70.71,45.74,158.0,-0.24,1.43,34.47,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-38.44,33.0,X-SC,4.53,91947.0,-36723.0,82017.0,-5.01,-28.54,89.20,35.20,219.0,-0.45,0.63,5.53,X40,NTT,FOOTWEAR
15,BSOFT,836.99,-14.02,60.0,H-SC,13.94,96696.0,-43957.0,116741.0,-1.94,-31.25,120.73,51.74,131.0,-0.38,0.67,11.87,XR,ATH,IT
66,SHALBY,327.00,1112.12,44.0,M-SC,21.83,164830.0,-16313.0,61778.0,-1.53,-9.01,37.48,25.10,235.0,-0.26,1.14,32.09,XY24,NTT,HEALTHCARE
80,UJJIVANSFB,60.00,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
86,VOLTAS,1530.00,1.58,54.0,X-MC,3.56,213090.0,21348.0,16408.0,-1.11,11.13,7.70,19.69,99.0,1.30,1.47,18.69,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.0,41.14,47.0,M-MC,9.65,223275.0,-1687.0,169332.0,-0.18,-0.75,75.84,74.53,192.0,-0.01,1.54,31.26,XY24,BTT,STEEL
80,UJJIVANSFB,60.0,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
36,IEX,219.0,-30.62,60.0,H-SC,16.46,198217.0,-1737.0,98970.0,-0.73,-0.87,49.93,48.63,115.0,-0.02,1.37,10.58,XR,NTT,MISC
88,WIPRO,420.0,-14.25,47.0,M-LC,5.76,150815.0,-130.0,109582.0,-0.27,-0.09,72.66,72.51,53.0,-0.00,1.04,5.93,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
79,TTKPRESTIG,770.0,99.36,57.0,M-SC,1.97,85903.0,-14874.0,14964.0,1.36,-14.76,17.42,0.09,245.0,-0.99,0.59,12.86,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.06,57.0,H-SC,2.67,227529.0,-42138.0,78475.0,-0.15,-15.63,34.49,13.47,138.0,-0.54,1.57,15.69,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-21.86,56.0,H-MC,7.02,105754.0,-26081.0,68307.0,-0.35,-19.78,64.59,32.03,98.0,-0.38,0.73,18.45,OX40N,NTT,IT
18,CERA,9475.0,-24.34,41.0,H-SC,2.62,138138.0,-37765.0,79789.0,0.15,-21.47,57.76,23.89,149.0,-0.47,0.95,19.30,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
57,RECLTD,446.00,45.36,54.0,M-LC,5.47,205535.0,2365.0,39771.0,0.00,1.16,19.35,20.74,55.0,0.06,1.42,8.11,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.24,48.0,M-LC,11.25,207930.0,2854.0,109725.0,-0.89,1.39,52.77,54.89,52.0,0.03,1.43,25.82,X5K,ATH,METALS
47,KPIGREEN,731.05,9.40,68.0,H-SC,5.28,130309.0,5012.0,53909.0,-0.98,4.00,41.37,47.03,141.0,0.09,0.90,63.16,MH,ATH,POWER
81,UNIONBANK,163.00,-8.53,65.0,M-LC,9.81,166059.0,25219.0,19279.0,0.75,17.91,11.61,31.59,66.0,1.31,1.14,49.14,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-16.77,75.0,H-MC,15.20,215505.0,35697.0,76720.0,0.29,19.85,35.60,62.52,88.0,0.47,1.49,60.71,XR,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.01,65.0,M-SC,3.15,128414.0,13616.0,80426.0,0.33,11.86,62.63,81.92,223.0,0.17,0.89,52.02,XR,NTT,DURABLES
47,KPIGREEN,731.05,9.40,68.0,H-SC,5.28,130309.0,5012.0,53909.0,-0.98,4.00,41.37,47.03,141.0,0.09,0.90,63.16,MH,ATH,POWER
80,UJJIVANSFB,60.00,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
37,INDIAMART,4810.62,-53.33,56.0,H-SC,6.89,130173.0,6837.0,124784.0,0.04,5.54,95.86,106.72,119.0,0.05,0.90,30.22,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-38.44,33.0,X-SC,4.53,91947.0,-36723.0,82017.0,-5.01,-28.54,89.20,35.20,219.0,-0.45,0.63,5.53,X40,NTT,FOOTWEAR
24,DMART,5391.45,-16.82,37.0,X-LC,10.01,101741.0,-845.0,27653.0,-0.44,-0.82,27.18,26.13,38.0,-0.03,0.70,24.54,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.07,39.0,X-MC,5.40,200678.0,331.0,30182.0,0.44,0.17,15.04,15.23,178.0,0.01,1.38,32.47,X40N,NTT,REALTY
53,PGHH,17907.65,-30.28,39.0,X-MC,2.90,201660.0,840.0,66951.0,0.28,0.42,33.20,33.76,80.0,0.01,1.39,5.64,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-1.97,49.0,H-LC,1.04,157470.0,-28625.0,76105.0,-0.33,-15.38,48.33,25.51,15.0,-0.38,1.09,16.04,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.23,68.0,X-LC,1.51,205747.0,5609.0,15287.0,0.02,2.80,7.43,10.44,4.0,0.37,1.42,8.52,X40,NTT,FMCG
53,PGHH,17907.65,-30.28,39.0,X-MC,2.90,201660.0,840.0,66951.0,0.28,0.42,33.20,33.76,80.0,0.01,1.39,5.64,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,41.0,X-MC,3.09,310506.0,-17485.0,124016.0,-0.94,-5.33,39.94,32.48,92.0,-0.14,2.14,5.49,X40,ATH,ELECTRICAL
86,VOLTAS,1530.00,1.58,54.0,X-MC,3.56,213090.0,21348.0,16408.0,-1.11,11.13,7.70,19.69,99.0,1.30,1.47,18.69,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COLPAL,3767.14,-5.42,42.0,X-MC,8.10,221020.0,-42345.0,155686.0,-0.28,-16.08,70.44,43.04,84.0,-0.27,1.52,2.61,XY25,ATH,FMCG
52,PAGEIND,51769.93,-27.54,39.0,X-MC,6.87,81850.0,-1660.0,21690.0,-0.15,-1.99,26.50,23.98,82.0,-0.08,0.56,3.72,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.48,46.0,X-MC,5.83,180203.0,-20303.0,117456.0,0.05,-10.13,65.18,48.45,90.0,-0.17,1.24,3.92,X40N,ATH,FINANCE
3,ACC,3906.00,-37.64,52.0,X-MC,2.69,185650.0,-52301.0,204958.0,-0.33,-21.98,110.40,64.15,174.0,-0.26,1.28,4.14,XY24,BTT,CEMENT
85,VBL,671.64,-19.44,47.0,X-LC,9.52,289971.0,-25871.0,138548.0,-1.06,-8.19,47.78,35.67,5.0,-0.19,2.00,4.53,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-43.25,57.0,X-SC,38.26,56097.0,-8909.0,159837.0,5.30,-13.70,284.93,232.18,198.0,-0.06,0.39,9.42,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.54,39.0,X-MC,6.87,81850.0,-1660.0,21690.0,-0.15,-1.99,26.50,23.98,82.0,-0.08,0.56,3.72,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.68,44.0,X-SC,4.66,82222.0,-62938.0,138865.0,0.55,-43.36,168.89,52.31,136.0,-0.45,0.57,10.06,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,44.0,X-MC,2.99,89625.0,-1053.0,15962.0,0.35,-1.16,17.81,16.44,101.0,-0.07,0.62,18.78,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-38.44,33.0,X-SC,4.53,91947.0,-36723.0,82017.0,-5.01,-28.54,89.20,35.20,219.0,-0.45,0.63,5.53,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.40,58.0,X-LC,11.63,289717.0,-56239.0,122927.0,-0.09,-16.26,42.43,19.28,1.0,-0.46,2.00,7.10,X40,ATH,IT
40,INFY,2275.00,-17.80,59.0,X-LC,6.55,320054.0,7166.0,164508.0,-0.13,2.29,51.40,54.87,3.0,0.04,2.21,10.76,X40,BTT,IT
42,ITC,452.00,-36.23,68.0,X-LC,1.51,205747.0,5609.0,15287.0,0.02,2.80,7.43,10.44,4.0,0.37,1.42,8.52,X40,NTT,FMCG
85,VBL,671.64,-19.44,47.0,X-LC,9.52,289971.0,-25871.0,138548.0,-1.06,-8.19,47.78,35.67,5.0,-0.19,2.00,4.53,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,54.0,H-LC,5.98,251856.0,-9763.0,128975.0,0.17,-3.73,51.21,45.57,7.0,-0.08,1.74,7.17,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7088.89,63.0,L-SC,7.45,78882.0,-14928.0,91543.0,0.24,-15.91,116.05,81.67,269.0,-0.16,0.54,52.76,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,47.0,L-SC,27.63,80544.0,-33005.0,73053.0,-0.62,-29.07,90.70,35.27,268.0,-0.45,0.56,97.13,XR,NTT,HOTELS
50,MASFIN,398.61,-20.05,46.0,H-SC,9.13,90705.0,-7275.0,28880.0,0.32,-7.42,31.84,22.05,152.0,-0.25,0.63,31.85,XR,ATH,FINANCE
87,WHIRLPOOL,2270.00,-38.01,65.0,M-SC,3.15,128414.0,13616.0,80426.0,0.33,11.86,62.63,81.92,223.0,0.17,0.89,52.02,XR,NTT,DURABLES
47,KPIGREEN,731.05,9.40,68.0,H-SC,5.28,130309.0,5012.0,53909.0,-0.98,4.00,41.37,47.03,141.0,0.09,0.90,63.16,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-198.86,82.0,M-SC,28.62,171441.0,22221.0,121963.0,0.99,14.89,71.14,96.62,208.0,0.18,1.18,85.54,XY25,NTT,FINANCE
5,ANGELONE,3033.00,18.37,72.0,X-SC,5.34,209830.0,18824.0,38881.0,-0.70,9.86,18.53,30.21,157.0,0.48,1.45,32.79,X40N,NTT,FINANCE
69,SONACOMS,806.63,-32.06,71.0,M-SC,8.30,84604.0,-16555.0,56558.0,-0.08,-16.37,66.85,39.54,202.0,-0.29,0.58,19.30,AR,ATH,AUTO
12,BANKINDIA,190.00,-16.77,75.0,H-MC,15.20,215505.0,35697.0,76720.0,0.29,19.85,35.60,62.52,88.0,0.47,1.49,60.71,XR,NTT,BANKS
71,SURYODAY,240.00,75.44,68.0,H-SC,14.12,160752.0,-18319.0,84041.0,0.95,-10.23,52.28,36.71,135.0,-0.22,1.11,59.50,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.33
1,25,43.41
2,50,74.21


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.84
LC    31.99
MC    24.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.79
X40      16.12
XY25     12.01
XR       11.29
X40N     10.47
AR        7.76
OX40N     7.44
X200      1.74
X5K       1.43
SR        1.08
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.25
X-LC    21.38
X-MC    18.50
M-SC    12.59
M-LC     5.03
X-SC     4.57
H-LC     4.57
H-MC     3.85
M-MC     1.54
L-SC     1.43
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-4.12,38.09
IT,12.42,-16.79,79.42
FINANCE,12.08,-7.95,56.88
MISC,6.84,-15.82,79.99
BANKS,6.64,-3.66,58.19
ELECTRICAL,5.73,-8.78,47.35
PAINTS,5.48,-16.10,33.41
INSURANCE,3.80,-0.58,38.08
AUTO,2.76,-43.57,103.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3255482.0,22
XR,1314303.0,14
AR,1165307.0,9
X40,818439.0,12
X40N,640689.0,9
OX40N,566287.0,10
XY25,439968.0,8
SR,265474.0,2
X5K,109725.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3336340.0,25
M-SC,1219207.0,15
X-MC,1193145.0,14
X-LC,1038439.0,13
X-SC,517676.0,6
H-MC,322935.0,3
M-LC,278357.0,4
H-LC,276050.0,3
L-SC,254203.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1142822.0      6
           AR         836740.0      5
           XR         777593.0      7
M-SC       XY24       697087.0      6
X-MC       XY24       582923.0      4
X-LC       X40        445932.0      5
           XY24       306294.0      2
X-MC       X40        290490.0      6
X-SC       X40N       275822.0      4
H-SC       SR         265474.0      2
           OX40N      259802.0      4
X-LC       X40N       217229.0      3
H-LC       AR         205080.0      2
H-MC       XY24       177908.0      1
X-MC       XY25       172094.0      2
M-MC       XY24       169332.0      1
L-SC       XR         164596.0      2
X-SC       XY24       159837.0      1
M-SC       OX40N      148571.0      4
X-MC       X40N       147638.0      2
M-SC       XR         128099.0      2
           AR         123487.0      2
           XY25       121963.0      1
M-LC       X5K        109725.0      1
           XR         109582.0      1
L-SC       OX40N       89607.0      1
X-SC       X40         82017.0      1
H-MC       XR          76720.0      1
H-LC       X200        70970.0      1
X-LC       XY25        68984.0      3
H-MC       OX40N       68307.0      1
L-MC       XR          57713.0      1
H-SC       MH          53909.0      1
M-LC       XY25        39771.0      1
L-LC       XY25        37156.0      1
M-LC       XY24        19279.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
